In [1]:
%load_ext autoreload
%autoreload 2

In [34]:
import pandas as pd
import numpy as np
import pyproj
from pathlib import Path
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt

In [3]:
raw_data_path = Path.joinpath(Path.cwd(),'raw')

In [8]:
aa = pd.read_excel(f'metadata.xls', skiprows=[0])
aa

,5099-2,4699,ALMA,Unnamed: 3,19.98,772159,246134,722.9,0,A916,...,-77.1,56,0.00481373,0.00365777,0.0274539,0.02161937,0.00481427,0.00365822,0.02746034,0.02162476
0,9409-0,331,YARKA,NaN,220.16,761737,216659,74.9,0,3b31,...,-81,56,0,0,0,0,0,0,0,0
1,9406-0,30494,THEATER PALACE KIRYAT MOTZKIN,NaN,345.37,750049,207905,9.0,0,r462,...,-81,56,0,0,0,0,0,0,0,0


In [7]:
aa = pd.ExcelFile(f'metadata.xls')
aa

In [10]:
aa.parse()

In [19]:
def cellcom_ids(site_id):
    ''' remove IP numbers from cellcom site_a_id/site_b_id,
    and also convert all letters to lower case'''
    if type(site_id) == float:
        return np.nan
    elif len(site_id.strip('.0123456789;')) == 0:
        return np.nan
    elif '; ' in site_id:
        return site_id.split('; ')[1]
    else:
        return site_id[0:4]
        
def process_cellcom(xlfile, col_names):
    ''' Process metadata for cellcom '''
#     xl = pd.ExcelFile(xlfile)

#     df = xl.parse('Sheet1', skiprows=1) # skip the first row of the excel file
    df = pd.read_excel(xlfile)
    #cols = ['LINK_NO', 'STATUS', '', 'TX_FREQ_HIGH_MHZ', 'TX_FREQ_LOW_MHZ', 'POL', 'LENGTH_KM', '', 'SITE1_NAME', 'ID_SITE1', 'EAST1', 'NORTH1', 'HEIGHT_ABOVE_SEA1_M', 'SITE2_NAME', 'ID_SITE2', 'EAST2', 'NORTH2', 'HEIGHT_ABOVE_SEA2_M','']
    cols = ['LINK_NO', 'STATUS', 'TX_FREQ_HIGH_MHZ', 'TX_FREQ_LOW_MHZ', 'POL', 'LENGTH_KM', 'SITE1_NAME', 'ID_SITE1', 'EAST1', 'NORTH1', 'HEIGHT_ABOVE_SEA1_M', 'SITE2_NAME', 'ID_SITE2', 'EAST2', 'NORTH2', 'HEIGHT_ABOVE_SEA2_M']
    df = df[cols]
    df.insert(16,'SLOTS', '')
    df.insert(0,'SP', 'cellcom')
    df.columns = col_names
    
    # convert EAST/NORTH to LAT/LON decimal
    bng = pyproj.Proj(init='epsg:2039')
    wgs84 = pyproj.Proj(init='epsg:4326')
    # lon, lat = pyproj.transform(from,to,easting,northing)
    df['LON1'],df['LAT1'] = pyproj.transform(bng, wgs84, df['LON1'].values, df['LAT1'].values)
    df['LON2'],df['LAT2'] = pyproj.transform(bng, wgs84, df['LON2'].values, df['LAT2'].values)
    
    # process cellcom ids to fix problems
    df['SITE1_ID'] = df['SITE1_ID'].apply(cellcom_ids)
    df['SITE2_ID'] = df['SITE2_ID'].apply(cellcom_ids)
    
    # remove '-X' from cml_id
    df['Link_num'] = df['Link_num'].str.partition('-')[0]
    return df


In [20]:
# process all the metadata 
col_names = ['SP', 'Link_num', 'Status', 'Frequency1', 
             'Frequency2', 'Polarization', 'Length_KM', 
             'SITE1_Name', 'SITE1_ID', 'LON1', 'LAT1', 
             'Height_above_sea1', 'SITE2_Name', 'SITE2_ID', 
             'LON2', 'LAT2', 'Height_above_sea2','SLOTS']
MD = process_cellcom(f'metadata.xls', col_names)

# convert object to numeric values with additional processing
#MD.loc[:,'Link_num'] = pd.to_string(MD.loc[:,'Link_num'], errors='coerce')

MD.loc[:,'Frequency1'] = pd.to_numeric(MD.loc[:,'Frequency1'], errors='coerce')*1e9/1000 # convert MHz to GHz
MD.loc[:,'Frequency2'] = pd.to_numeric(MD.loc[:,'Frequency2'], errors='coerce')*1e9/1000  

MD.loc[:,'LAT1'] = pd.to_numeric(MD.loc[:,'LAT1'], errors='coerce')
MD.loc[:,'LON1'] = pd.to_numeric(MD.loc[:,'LON1'], errors='coerce')
MD.loc[:,'LAT2'] = pd.to_numeric(MD.loc[:,'LAT2'], errors='coerce')
MD.loc[:,'LON2'] = pd.to_numeric(MD.loc[:,'LON2'], errors='coerce')
MD.loc[:,'Length_KM'] = pd.to_numeric(MD.loc[:,'Length_KM'], errors='coerce')

MD.loc[:,'Height_above_sea1'] = pd.to_numeric(MD.loc[:,'Height_above_sea1'], errors='coerce')
MD.loc[:,'Height_above_sea2'] = pd.to_numeric(MD.loc[:,'Height_above_sea2'], errors='coerce')

Index(['LINK_NO', 'SITE1', 'SITE1_NAME', 'SHTAHIM_SITE1', 'AZIMUTH1', 'NORTH1',
       'EAST1', 'HEIGHT_ABOVE_SEA1_M', 'SAU_SITE1', 'ID_SITE1',
       'TX_FREQ_HIGH_MHZ', 'ANTENA1', 'ANTENAHEIGHT_1', 'BUILDING_HEIGHT1_M',
       'SPLITTER_WAVEGUIDE1', 'SITE2', 'SITE2_NAME', 'SHTAHIM_SITE2',
       'AZIMUTH2', 'NORTH2', 'EAST2', 'HEIGHT_ABOVE_SEA2_M', 'SAU_SITE2',
       'ID_SITE2', 'TX_FREQ_LOW_MHZ', 'ANTENA2', 'ANTENAHEIGHT_2',
       'BUILDING_HEIGHT2_M', 'SPLITTER_WAVEGUIDE2', 'SUPPLIER_NAME',
       'BAND_GHZ', 'RADIO_INDEX', 'POL', 'LENGTH_KM', 'PROTECTED',
       'CAPACITY_MBPS', 'BW_MHZ', 'AGILE', 'TYPE_APPROVAL',
       'MAX_OUTPUT_POWER1_DBM', 'MAX_OUTPUT_POWER2_DBM',
       'ACTUAL_TX_POWER1_DBM', 'ACTUAL_TX_POWER2_DBM',
       'PLANNED_RSL_DBM(SITE A)', 'PLANNED_RSL_DBM(SITE B)', 'ACTUAL_RSL_DBM',
       'PLANNED_FADE_MARGIN1_DB', 'PLANNED_FADE_MARGIN2_DB',
       'PLANNED_EFF_FADE_MARGIN1_DB', 'PLANNED_EFF_FADE_MARGIN2_DB', 'SUBNET',
       'COW', 'SPACE_DIV Site1', 'SPACE_

/Users/adameshel/opt/anaconda3/envs/process_cml_rawdata_env/lib/python3.8/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Users/adameshel/opt/anaconda3/envs/process_cml_rawdata_env/lib/python3.8/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/var/folders/3d/dzs676gd66d1hmjz_1_jptjc0000gn/T/ipykernel_1821/2781794087.py:31: DeprecationWarning: This function is deprecated.

In [30]:
# select raw-data files to open
only_files = sorted([f for f in listdir(raw_data_path) if isfile(join(raw_data_path, f))])
# only_files = ['cellcom/SOEM-M_HC_RADIO_SINK_20150121_001500.txt',
#              'cellcom/SOEM-M_HC_RADIO_SOURCE_20150121_001500.txt'] 

In [91]:
# select specific links (by link number from cellcom meta-data file)
sel_links = ['5099','9409','9406']

to_process = MD.loc[MD['Link_num'].isin(sel_links)] # take only metadata for links in sel_links

sel_links = [i.split('-', 1)[0] for i in sel_links] # take only number before the '-' in each link number
    
# find raw-data for the links
RD_rx = [] # gather all RADIO_SINK
RD_tx = [] # gather all RADIO_SOURCE

for rdfile in only_files[0:1]:
    rdfile = str(raw_data_path.joinpath(rdfile))
    RD = pd.read_csv(rdfile,  index_col = False)
    RD.insert(6,'Site', '')
    RD['Site'] = RD['NeAlias'].str.partition('_')[0]
    RD['Site'] = RD['Site'].str.lower()
    RD['NeAlias'] = RD['NeAlias'].str.rpartition('_')[2]
    RD['NeAlias'] = RD['NeAlias'].str.rpartition('.')[0]
    
    # find specific links in the file
    RD = RD.loc[RD['NeAlias'].isin(sel_links)]
    
    # separate to RX and TX
    if str.find(rdfile, 'RADIO_SINK') != -1:
        RD = RD[['Time','Interval','Site','NeAlias','PowerRLTMmin','PowerRLTMmax']]
        RD_rx.append(RD)
        
    elif str.find(rdfile, 'RADIO_SOURCE') != -1:
        RD = RD[['Time','Interval','Site','NeAlias','PowerTLTMmin','PowerTLTMmax']]
        RD_tx.append(RD)

RD_rx = pd.concat(RD_rx) # the min/max RSL
RD_tx = pd.concat(RD_tx) # the min/max TSL

# replace NeAlias with link_number
RD_rx = RD_rx.rename(columns={'NeAlias': 'Hop_number', 'Site': 'Measuring_site'})
RD_tx = RD_tx.rename(columns={'NeAlias': 'Hop_number', 'Site': 'Measuring_site'})

# take only 15 minute data
RD_rx = RD_rx[RD_rx['Interval'] == 15]
RD_tx = RD_tx[RD_tx['Interval'] == 15]

# If you want you can select RSL and TSL for a specific link
RSL = RD_rx[RD_rx['Link_number'] == '6446']
TSL = RD_tx[RD_tx['Link_number'] == '6446']

# Take only time and min/max RSL or TSL
RSL = RSL[['Time','PowerRLTMmin','PowerRLTMmax']]
TSL = TSL[['Time','PowerTLTMmin','PowerTLTMmax']]
print('Done')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/adameshel/Documents/Python_scripts/process_cml_rawdata/raw/SOEM-M_HC_RADIO_SINK_20131201_001500.txt'

In [66]:
RD_tx

,Time,Interval,Measuring_site,Link_number,PowerTLTMmin,PowerTLTMmax
